In [108]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import matplotlib.pyplot as plt

In [109]:
class Cora():
    def __init__(self, path, dataset):
        self.path = path
        self.dataset = dataset

    def process_data(self):
        id_feature_label = np.genfromtxt('{path}{dataset}.content'
                                    .format(path=self.path, dataset=self.dataset)
                                    , dtype=np.dtype(str))
        
        feature = sp.csc_matrix(id_feature_label[:, 1:-1], dtype=np.float32)

        label = np.array(pd.get_dummies(id_feature_label[:, -1]), dtype=np.int32)

        adj = self.build_adjcency_matrix(id_feature_label, label)

        adj = adj + adj.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

        def normalize(adj):
            adj += sp.eye(adj.shape[0])
            row_sum = np.array(adj.sum(1))
            d_hat = np.power(row_sum, -0.5).flatten()
            d_hat[np.isinf(d_hat)] = 0.
            d_hat = sp.diags(d_hat)
            return d_hat.dot(adj).dot(d_hat)
        
        adj = normalize(adj)

        index_train = range(140)
        index_val = range(140, 640)
        index_test = range(1708, 2708)

        def sample_mask(index, length):
            mask = np.zeros(length)
            mask[index] = 1
            return np.array(mask, dtype=np.bool_)
        
        length = label.shape[0]

        train_mask = sample_mask(index_train, length)
        val_mask = sample_mask(index_val, length)
        test_mask = sample_mask(index_test, length)

        # print(feature.shape, type(feature))
        # print(label.shape, type(label))
        # print(adj.shape, type(adj))
        # print(train_mask.shape, type(train_mask))
        # print(val_mask.shape, type(val_mask))
        # print(test_mask.shape, type(test_mask))

        feature = torch.FloatTensor(np.array(feature.todense()))
        label = torch.LongTensor(np.where(label)[1])

        def sparse_mx_to_torch_sparse_tensor(sparse_mx):
            """Convert a scipy sparse matrix to a torch sparse tensor."""
            sparse_mx = sparse_mx.tocoo().astype(np.float32)
            indices = torch.from_numpy(
                np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
            values = torch.from_numpy(sparse_mx.data)
            shape = torch.Size(sparse_mx.shape)
            return torch.sparse.FloatTensor(indices, values, shape)

        adj = sparse_mx_to_torch_sparse_tensor(adj)

        train_mask = torch.from_numpy(train_mask)
        val_mask = torch.from_numpy(val_mask)
        test_mask = torch.from_numpy(test_mask)
    
        return feature, label, adj, train_mask, val_mask, test_mask
    

    def build_adjcency_matrix(self, id_feature_label, label):
        id = np.array(id_feature_label[:, 0], dtype=np.int32)
        id_map = {j: i for i, j in enumerate(id)}

        edges = np.genfromtxt("{path}{dataset}.cites".format(path=self.path, dataset=self.dataset)
                                , dtype=np.int32)

        edges_order_by_id = np.array(list(map(id_map.get, edges.flatten())), dtype=np.int32) \
                            .reshape(edges.shape)

        row = edges_order_by_id[:, 0]
        col = edges_order_by_id[:, 1]
        data = np.ones(edges_order_by_id.shape[0])
        adj = sp.coo_matrix((data, (row, col))
                                        , shape=(label.shape[0], label.shape[0])
                                        , dtype=np.float32)
        return adj
    

In [110]:
cora = Cora(path='../../data_processing/data/cora/', dataset='cora')
cora.process_data()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([2, 5, 4,  ..., 1, 0, 2]),
 tensor(indices=tensor([[   0,   14,  258,  ..., 2705, 2706, 2707],
                        [   0,    0,    0,  ..., 2705, 2706, 2707]]),
        values=tensor([0.2500, 0.1118, 0.1443,  ..., 1.0000, 0.2000, 0.2500]),
        size=(2708, 2708), nnz=8137, layout=torch.sparse_coo),
 tensor([ True,  True,  True,  ..., False, False, False]),
 tensor([False, False, False,  ..., False, False, False]),
 tensor([False, False, False,  ...,  True,  True,  True]))

In [111]:
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim, use_bias=True):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.use_bias = use_bias
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        if self.use_bias:
            self.bias = nn.Parameter(torch.Tensor(output_dim))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    
    def reset_parameters(self):
        init.kaiming_normal_(self.weight)
        if self.use_bias:
            init.zeros_(self.bias)
    

    def forward(self, adjacency, input_feature):
        support = torch.mm(input_feature, self.weight)
        output = torch.sparse.mm(adjacency, support)
        if self.use_bias:
            return output + self.use_bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.input_dim) + ' -> ' \
            + str(self.output_dim) + ')'


In [112]:
class GCN(nn.Module):
    def __init__(self, input_dim=1433):
        super(GCN, self).__init__()
        self.gcn1 = GCN(input_dim, 16)
        self.gcn2 = GCN(16, 7)

    def forward(self, adjacency, feature):
        h = F.relu(self.gcn1(adjacency, feature))
        logits = self.gcn2(adjacency, h)
        return logits
    

In [113]:
# 定义超参数
LEARNING_RATE = 0.1 # 学习率
WEIGHT_DACAY = 5e-4 # 权重衰减
EPOCHS = 200
DEVICE = 'cpu'

In [114]:
feature, label, adj, train_mask, val_mask, test_mask = Cora(path='../../data_processing/data/cora/', dataset='cora').process_data()

print(feature.shape, type(feature))
print(label.shape, type(label))
print(adj.shape, type(adj))
print(train_mask.shape, type(train_mask))
print(val_mask.shape, type(val_mask))
print(test_mask.shape, type(test_mask))

input_dim = 1433


torch.Size([2708, 1433]) <class 'torch.Tensor'>
torch.Size([2708]) <class 'torch.Tensor'>
torch.Size([2708, 2708]) <class 'torch.Tensor'>
torch.Size([2708]) <class 'torch.Tensor'>
torch.Size([2708]) <class 'torch.Tensor'>
torch.Size([2708]) <class 'torch.Tensor'>


In [115]:
model = GCN(input_dim).to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), 
                       lr=LEARNING_RATE, 
                       weight_decay=WEIGHT_DACAY)


TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given